In [6]:
import os

# 1. Extract Keypoints
load video data and extract keypoints using mediapose into /data/pose_data.csv

In [7]:
from src import extract_keypoints as extract

# Process all MP4s from a folder and extract labeled keypoints
video_dir = os.path.join("data", "raw")
df = extract.process_directory(video_dir)

# Save result for training
csv_path = os.path.join("data", "pose_data.csv")
extract.save_dataframe(df, csv_path)


🔍 Processing left_1.mp4...
🔍 Processing left_2.mp4...
🔍 Processing left_3.mp4...
🔍 Processing neutral_1.mp4...
🔍 Processing neutral_2.mp4...
🔍 Processing neutral_3.mp4...
🔍 Processing right_1.mp4...
🔍 Processing right_2.mp4...
🔍 Processing right_3.mp4...
✅ Saved keypoints to data\pose_data.csv


# 2. Train Model
create classifier for the keypoints and save the model in models/tf_model

In [8]:
from src import train_pose_model as train
from sklearn.model_selection import train_test_split

model_dir = os.path.join("models", "tf_model")
label_map_path = os.path.join("models", "label_map.json")

# Load data
X, y = train.load_pose_data(csv_path)
y_encoded, label_encoder = train.encode_labels(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build and train model
model = train.build_model(input_dim=X.shape[1], num_classes=len(label_encoder.classes_))
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Save model and labels
train.save_model(model, model_dir)
train.save_label_map(label_encoder, label_map_path)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,475 (99.51 KB)

 Trainable params: 25,475 (99.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5650 - loss: 0.9654 - val_accuracy: 0.8995 - val_loss: 0.3905
Epoch 2/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8956 - loss: 0.4138 - val_accuracy: 0.8995 - val_loss: 0.3223
Epoch 3/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8856 - loss: 0.3721 - val_accuracy: 0.8995 - val_loss: 0.3341
Epoch 4/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9047 - loss: 0.3677 - val_accuracy: 0.8995 - val_loss: 0.3017
Epoch 5/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9136 - loss: 0.2966 - val_accuracy: 0.8995 - val_loss: 0.2944
Epoch 6/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9020 - loss: 0.3001 - val_accuracy: 0.8995 - val_loss: 0.2986
Epoch 7/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9057 - loss: 0.3199 - val_accuracy: 0.8995 - val_loss: 0.2810
Epoch 8/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9098 - loss: 0.2868 - val_accuracy: 0.8995 - val_loss:

INFO:tensorflow:Assets written to: models\tf_model\assets
